- Sentence Embedding using Sentence-BERT  
- Similarity based answering

In [ ]:
! pip install sentence_transformers

In [ ]:
import numpy as np
import pandas as pd
import urllib.request
from numpy import dot
from numpy.linalg import norm

from sentence_transformers import SentenceTransformer

In [ ]:
# 챗봇 데이터를 로드 

urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")

dataset = pd.read_csv('ChatBotData.csv')

In [ ]:
dataset.head(10)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.,0
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.,0


In [ ]:
dataset.label.unique()

array([0, 1, 2])

In [ ]:
dataset.loc[dataset.label == 0]

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
5285,힘내야지,응원합니다!,0
5286,힘든 것 좀 끝났으면,다 지나갈 거예요.,0
5287,힘든 시기가 지나갔으면,다 지나갈 거예요.,0
5288,힘든 데도 날 도와준 사람,잊지말고 보답하세요.,0


### Pretrained BERT

https://www.sbert.net/docs/pretrained_models.html

Sentence Natural Language Inference : 문장 사이 관계를 파악  
- 0 (entailment)  
- 1 (neutral)  
- 2 (contradiction)  

Semantic Textual Similarity : 문장 사이의 유사도 측정  
- 문장 쌍 & 유사도 점수  

로 학습한 모델


In [ ]:
bert_model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

`Basic Method`  

질문을 입력하면,  
예시 데이터셋 (Question-Answer 쌍)의 Question과 가장 유사한 Q를 찾음.  
(embedding vector의 cosine similarity)  

해당 Q과 매치되는 A를 반환  

In [ ]:
dataset['Q_vector'] = dataset.apply(lambda x : bert_model.encode(x.Q), axis=1)

In [ ]:
dataset.loc[0, 'Q_vector'].shape

(768,)

In [ ]:
dataset

,Q,A,label,Q_vector
0,12시 땡!,하루가 또 가네요.,0,"[0.20179531, -0.034437913, 1.5395736, 0.010697..."
1,1지망 학교 떨어졌어,위로해 드립니다.,0,"[0.077166006, -0.0342782, 0.8624418, 0.0263605..."
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0,"[0.10445242, -0.012432262, 1.0132883, 0.022501..."
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0,"[0.09760745, -0.046716794, 0.89369446, 0.02104..."
4,PPL 심하네,눈살이 찌푸려지죠.,0,"[-0.07002864, 0.03196112, 1.491543, 4.3385797e..."
...,...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2,"[0.15363254, -0.32605517, 0.6987688, 0.1091404..."
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2,"[0.15363254, -0.32605517, 0.6987688, 0.1091404..."
11820,흑기사 해주는 짝남.,설렜겠어요.,2,"[-0.03608489, -0.0052648424, 0.8913545, -0.011..."
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2,"[-0.084686615, -0.14302953, 0.115261056, 0.386..."


In [ ]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
def QA(question): 
    embedding = bert_model.encode(question)
    dataset['score'] = dataset.apply(lambda x: cos_sim(x['Q_vector'], embedding), axis=1)

    return dataset.nlargest(n=5, columns=['score']).A


In [ ]:
ques = input("챗봇아 ")
print(QA(ques))

챗봇아 안녕
2949        안녕하세요.
2950        안녕하세요.
6901        안녕하세요.
5299    시간이 무색하네요.
8221     맘고생 많았어요.
Name: A, dtype: object


In [ ]:
ques = input("챗봇아 ")
print(QA(ques))

챗봇아 속상해
2532        기분전환을 해보세요.
5576           괴로울만 해요.
6791        마음이 복잡했나봐요.
6884     진통제 한 알 먹어보세요.
3141    스트레스가 더 큰 적이에요.
Name: A, dtype: object


In [ ]:
ques = input("챗봇아 ")
print(QA(ques))

챗봇아 똑땅해
2251     한가지만 보면 몰라요.
6561     한가지만 보면 몰라요.
322     괜찮은 선택이길 바라요.
5564        울어도 괜찮아요.
2021          저랑 놀아요.
Name: A, dtype: object


In [ ]:
dataset[dataset['Q'].str.contains("똑땅")]

,Q,A,label,Q_vector,score
